# 기본 설정

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS as stopwords
from PIL import Image, ImageFilter
from wordcloud import ImageColorGenerator
from matplotlib import font_manager, rc
from collections import Counter
import konlpy
from konlpy.tag import Mecab
import time
import seaborn as sns
import matplotlib.pyplot as plt
from random import shuffle
import json
from tqdm.notebook import tqdm
tqdm.pandas()
import gensim 
from gensim.models import Word2Vec, FastText, Doc2Vec
import fasttext
from bokeh.plotting import figure, output_file, save
from bokeh.io import curdoc, show
from bokeh.transform import factor_cmap, factor_mark
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
from sklearn.feature_extraction.text import TfidfVectorizer
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
output_notebook()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Loading BokehJS ...

In [9]:
bills = pd.read_csv('/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학 이후/코딩 관련/개인프로젝트_국회의원 추천 (20대)/1차 데이터 및 jupyter/bills_24839_all_cleansed.csv')
senators = pd.read_csv('/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학 이후/코딩 관련/개인프로젝트_국회의원 추천 (20대)/1차 데이터 및 jupyter/(원본) assembly_memebers_295.csv')

In [7]:
senators_w_clusters = pd.read_csv("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학 이후/코딩 관련/개인프로젝트_국회의원 추천 (20대)/1차 데이터 및 jupyter/senator with cluster_13 ver21.0.csv")
senators_w_clusters.rename(columns={'index':'한글이름'}, inplace=True)

In [10]:
senators['한글이름'] = senators['이름'].str.split(' ').str[0]
senators['한자이름'] = senators['이름'].str.split(' ').str[1]

In [11]:
bills = bills[bills['발의자'] != '위원장']
bills = bills[bills['발의자'] != '정부'] 
bills = bills.dropna()

# methods

In [16]:
def count_bills(name, bills_df):
    df_temp = bills_df[bills_df['발의자'].str.contains(name)]
    bills_deleted = df_temp[(df_temp['상태']=='폐기') | (df_temp['상태']=='철회')].shape[0]
    df_temp_save = df_temp[~(df_temp['상태']=='폐기') & ~(df_temp['상태']=='철회')]
    bills_일부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('일부개정법률안')].shape[0]
    bills_전부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('전부개정법률안')].shape[0]
    bills_제정법안 = df_temp_save[(~df_temp_save['법안명'].str.contains('일부개정법률안'))& (~df_temp_save['법안명'].str.contains('전부개정법률안'))].shape[0]
    dict_ = {'일부개정법률안 비율': str(round((bills_일부개정법률안/df_temp.shape[0])*100,2))+'%',
            '전부개정법률안 비율': str(round((bills_전부개정법률안/df_temp.shape[0])*100,2))+'%',
            '제정법률안 비율': str(round((bills_제정법안/df_temp.shape[0])*100,2))+"%",
            '폐기법률안 비율': str(round((bills_deleted/df_temp.shape[0])*100,2))+'%'}
    return dict_

def count_bills_all(bills_df):
    df_temp = bills_df
    bills_deleted = df_temp[(df_temp['상태']=='폐기') | (df_temp['상태']=='철회')].shape[0]
    df_temp_save = df_temp[~(df_temp['상태']=='폐기') & ~(df_temp['상태']=='철회')]
    bills_일부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('일부개정법률안')].shape[0]
    bills_전부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('전부개정법률안')].shape[0]
    bills_제정법안 = df_temp_save[(~df_temp_save['법안명'].str.contains('일부개정법률안'))& (~df_temp_save['법안명'].str.contains('전부개정법률안'))].shape[0]
    dict_ = {'일부개정법률안 비율': str(round((bills_일부개정법률안/df_temp.shape[0])*100,2))+'%',
            '전부개정법률안 비율': str(round((bills_전부개정법률안/df_temp.shape[0])*100,2))+'%',
            '제정법률안 비율': str(round((bills_제정법안/df_temp.shape[0])*100,2))+"%",
            '폐기법률안 비율': str(round((bills_deleted/df_temp.shape[0])*100,2))+'%'}
    return dict_


def get_quantile(senator):
    result = result
    return result

# 전체 distribution 분포 살펴보기

In [17]:
dict_bills_portion = {}
for senator in senators['한글이름']:
    dict_bills_portion[senator] = count_bills(senator, bills)

In [18]:
df_bills_portion = pd.DataFrame(dict_bills_portion).T

In [19]:
df_bills_portion['일부개정법률안 비율'] = df_bills_portion['일부개정법률안 비율'].str[:-1]
df_bills_portion['전부개정법률안 비율'] = df_bills_portion['전부개정법률안 비율'].str[:-1]
df_bills_portion['폐기법률안 비율'] = df_bills_portion['폐기법률안 비율'].str[:-1]
df_bills_portion['제정법률안 비율'] = df_bills_portion['제정법률안 비율'].str[:-1]

In [20]:
df_bills_portion['일부개정법률안 비율'] = df_bills_portion['일부개정법률안 비율'].astype(float)
df_bills_portion['전부개정법률안 비율'] = df_bills_portion['전부개정법률안 비율'].astype(float)
df_bills_portion['폐기법률안 비율'] = df_bills_portion['폐기법률안 비율'].astype(float)
df_bills_portion['제정법률안 비율'] = df_bills_portion['제정법률안 비율'].astype(float)

In [21]:
df_bills_portion['제정및전부개정법률안 비율'] = df_bills_portion['제정법률안 비율'] + df_bills_portion['전부개정법률안 비율']

# 상위 % 잘라낸 뒤 추천하기

In [22]:
df_bills_portion.reset_index(inplace=True)

In [23]:
senators = senators.merge(df_bills_portion, left_on="한글이름", right_on="index", how="left")
senators = senators.drop(columns=['index'])

In [24]:
df_temp = senators[['이름', '한글이름','제정및전부개정법률안 비율']].sort_values(['제정및전부개정법률안 비율'])

In [25]:
df_temp['순위'] = range(1, df_temp.shape[0]+1)

In [26]:
df_temp['percentile'] = round(100-(df_temp['순위']/295) * 100, 5)

In [27]:
df_temp['percentile'] = df_temp['percentile'].astype(str)
df_temp['percentile'] = df_temp['percentile'] + '%'
df_temp

,이름,한글이름,제정및전부개정법률안 비율,순위,percentile
284,허윤정 許允貞,허윤정,0.00,1,99.66102%
245,정은혜 鄭恩惠,정은혜,0.00,2,99.32203%
248,정점식 鄭点植,정점식,2.63,3,98.98305%
271,최도자 崔道子,최도자,3.89,4,98.64407%
224,임재훈 林哉勳,임재훈,4.32,5,98.30508%
14,권칠승 權七勝,권칠승,4.38,6,97.9661%
181,이규희 李揆熙,이규희,4.41,7,97.62712%
220,이후삼 李厚三,이후삼,4.42,8,97.28814%
15,금태섭 琴泰燮,금태섭,4.44,9,96.94915%
136,신용현 申容賢,신용현,4.49,10,96.61017%


# Calculate senators' bills index

In [29]:
def count_senators(data):
    data = [x for x in data if '/' in x]
    return len(data)

In [30]:
dict_count_biils = {}
for row in senators.itertuples():
    dict_temp = {}
    dict_temp['한글이름'] = row[-7]
    
    # Get the dataframe
    df_temp = bills[bills['발의자'].str.contains(row[-7])]
    bills_전부개정법률안 = df_temp[df_temp['법안명'].str.contains('전부개정법률안')]
    bills_제정법안 = df_temp[(~df_temp['법안명'].str.contains('일부개정법률안'))& (~df_temp['법안명'].str.contains('전부개정법률안'))]
    bills_total = pd.concat([bills_제정법안, bills_전부개정법률안])
    
    # count mean senators proposing 
    bills_total['발의자 수'] = bills_total['발의자'].str.split(' ').map(count_senators)
    
    # save in dictionary
    dict_temp['전체 발의법안 수'] = bills_total.shape[0]
    if dict_temp['전체 발의법안 수'] == 0:
        dict_temp['평균 발의자 수'] = 0.000000000000009
    else:    
        dict_temp['평균 발의자 수'] = round(bills_total['발의자 수'].mean())

    dict_count_biils[row[-6]] = dict_temp

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [32]:
df_result = pd.DataFrame(dict_count_biils).T.reset_index().rename(columns={'index':'한자이름'})
df_result['법안수/평균 발의자 수'] = df_result['전체 발의법안 수'] / df_result['평균 발의자 수'] 
df_result.sort_values(by='법안수/평균 발의자 수')
df_result['bills_index'] = df_result['법안수/평균 발의자 수'].astype(float)
df_result['bills_index_scaled'] = scaler.fit_transform(df_result[['bills_index']])
df_result

,한자이름,한글이름,전체 발의법안 수,평균 발의자 수,법안수/평균 발의자 수,bills_index,bills_index_scaled
0,姜吉夫,강길부,23,67,0.343284,0.343284,0.031618
1,姜炳遠,강병원,94,38,2.47368,2.473684,0.227839
2,姜錫振,강석진,61,47,1.29787,1.297872,0.119541
3,姜碩鎬,강석호,63,47,1.34043,1.340426,0.123460
4,姜昌一,강창일,78,38,2.05263,2.052632,0.189058
5,姜孝祥,강효상,38,63,0.603175,0.603175,0.055556
6,姜勳植,강훈식,116,34,3.41176,3.411765,0.314241
7,慶大秀,경대수,67,43,1.55814,1.558140,0.143513
8,高榕禛,고용진,75,34,2.20588,2.205882,0.203173
9,郭大勳,곽대훈,74,46,1.6087,1.608696,0.148169


In [33]:
bills_index = df_result.copy()
bills_index

,한자이름,한글이름,전체 발의법안 수,평균 발의자 수,법안수/평균 발의자 수,bills_index,bills_index_scaled
0,姜吉夫,강길부,23,67,0.343284,0.343284,0.031618
1,姜炳遠,강병원,94,38,2.47368,2.473684,0.227839
2,姜錫振,강석진,61,47,1.29787,1.297872,0.119541
3,姜碩鎬,강석호,63,47,1.34043,1.340426,0.123460
4,姜昌一,강창일,78,38,2.05263,2.052632,0.189058
5,姜孝祥,강효상,38,63,0.603175,0.603175,0.055556
6,姜勳植,강훈식,116,34,3.41176,3.411765,0.314241
7,慶大秀,경대수,67,43,1.55814,1.558140,0.143513
8,高榕禛,고용진,75,34,2.20588,2.205882,0.203173
9,郭大勳,곽대훈,74,46,1.6087,1.608696,0.148169


# Compare bills index within clusters

In [35]:
bills_index

,한자이름,한글이름,전체 발의법안 수,평균 발의자 수,법안수/평균 발의자 수,bills_index,bills_index_scaled
0,姜吉夫,강길부,23,67,0.343284,0.343284,0.031618
1,姜炳遠,강병원,94,38,2.47368,2.473684,0.227839
2,姜錫振,강석진,61,47,1.29787,1.297872,0.119541
3,姜碩鎬,강석호,63,47,1.34043,1.340426,0.123460
4,姜昌一,강창일,78,38,2.05263,2.052632,0.189058
5,姜孝祥,강효상,38,63,0.603175,0.603175,0.055556
6,姜勳植,강훈식,116,34,3.41176,3.411765,0.314241
7,慶大秀,경대수,67,43,1.55814,1.558140,0.143513
8,高榕禛,고용진,75,34,2.20588,2.205882,0.203173
9,郭大勳,곽대훈,74,46,1.6087,1.608696,0.148169


In [40]:
senators_w_clusters

,한글이름,x,y,정당,소속위원회,cluster_13
0,강길부,-7.843739,9.154598,무소속,산업통상자원중소벤처기업위원회,9
1,강병원,-11.518075,6.042161,더불어민주당,기획재정위원회,0
2,강석진,5.722982,-15.073046,미래통합당,농림축산식품해양수산위원회,10
3,강석호,8.049164,6.404850,미래통합당,농림축산식품해양수산위원회,4
4,강창일,3.764485,-7.717409,더불어민주당,행정안전위원회,1
5,강효상,14.575323,-3.458097,미래통합당,공공부문 채용비리 의혹과 관련된 국정조사특별위원회,11
6,강훈식,1.413982,2.741007,더불어민주당,국토교통위원회,7
7,경대수,5.061145,-14.301801,미래통합당,농림축산식품해양수산위원회,10
8,고용진,2.713527,-0.616596,더불어민주당,국회운영위원회,7
9,곽대훈,7.267257,0.719697,미래통합당,산업통상자원중소벤처기업위원회,4


In [48]:
df_merged = senators_w_clusters[['한글이름', '정당','cluster_13']].merge(bills_index[['한글이름', 'bills_index','bills_index_scaled']], on='한글이름', how='left')
df_merged

,한글이름,정당,cluster_13,bills_index,bills_index_scaled
0,강길부,무소속,9,0.343284,0.031618
1,강병원,더불어민주당,0,2.473684,0.227839
2,강석진,미래통합당,10,1.297872,0.119541
3,강석호,미래통합당,4,1.340426,0.123460
4,강창일,더불어민주당,1,2.052632,0.189058
5,강효상,미래통합당,11,0.603175,0.055556
6,강훈식,더불어민주당,7,3.411765,0.314241
7,경대수,미래통합당,10,1.558140,0.143513
8,고용진,더불어민주당,7,2.205882,0.203173
9,곽대훈,미래통합당,4,1.608696,0.148169


In [50]:
df_merged.groupby('cluster_13')[['bills_index']].mean()

,bills_index
cluster_13,
0,1.681261
1,2.250452
2,1.465430
3,4.385234
4,0.996764
5,1.338190
6,1.658192
7,2.490697
8,2.483238


In [51]:
df_merged.groupby('cluster_13')[['bills_index_scaled']].mean()

,bills_index_scaled
cluster_13,
0,0.154853
1,0.207278
2,0.134974
3,0.403903
4,0.091807
5,0.123254
6,0.152728
7,0.229406
8,0.228719


In [55]:
df_merged['bills_index_scaled'].mean()

0.17130570797659025

In [62]:
df_merged[df_merged['cluster_13']==5]

,한글이름,정당,cluster_13,bills_index,bills_index_scaled
28,김민기,더불어민주당,5,2.368421,0.218144
49,김승희,미래통합당,5,2.571429,0.236842
80,나경원,미래통합당,5,0.582090,0.053614
83,도종환,더불어민주당,5,0.871795,0.080297
84,맹성규,더불어민주당,5,1.217391,0.112128
90,박경미,더불어민주당,5,1.681818,0.154904
133,송희경,미래통합당,5,1.750000,0.161184
137,신상진,미래통합당,5,0.912281,0.084026
145,안민석,더불어민주당,5,2.105263,0.193906
151,염동열,미래통합당,5,0.892857,0.082237


전체 입법지수(bills_index_scaled)의 평균을 살펴보면 0.1713이다.  

### cluster 1
cluster 1의 입법지수는 0.15로 전체 평균보다는 낮은 수준이다.  
소속의원: 유승민, 박영선, 오신환, 이혜훈, 이철회 및 정의당 의원 다수  

### cluster 4
cluster 4의 입법지수는 0.403으로 전체보다 2배 이상 높은 것을 확인할 수 있었다.  
소속의원: 이언주, 정동영, 이종걸, 이찬열, 황주홍  

### cluster 6
cluster 6의 입법 지수는 0.1527로 전체 평균보다는 낮은 수준이다.  
소속의원: 나경원, 유은혜, 채이배, 이태규 